In [1]:
import pandas as pd
import numpy as np

In [22]:
df_all = pd.read_csv('df_all_fillna.csv')
df_all.tail(1)

,id,product_uid,product_title,search_term,relevance,product_description,brand,search_term_new
240759,240760,224428,Bosch 4 in. Bi-Metal Hole Saw,4 inch hole saw,NaN,The Bosch quick change bi-metal hole saws feat...,Bosch,4 inch hole saw


In [3]:
product_col = ['product_uid','product_title','product_description','brand']
df_product = df_all[product_col]

In [4]:
df_all.product_uid.unique().size

124428

In [96]:
product_all = df_product.drop_duplicates(subset = 'product_uid').reset_index(drop=True)
product_all.columns.values[0] = 'prod_id'
product_all.head()

,prod_id,product_title,product_description,brand
0,100001,Simpson Strong-Tie 12-Gauge Angle,"Not only do angles make joints stronger, they ...",Simpson Strong-Tie
1,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,BEHR Premium Textured DECKOVER is an innovativ...,BEHR Premium Textured DeckOver
2,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,Update your bathroom with the Delta Vero Singl...,Delta
3,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,Achieving delicious results is almost effortle...,Whirlpool
4,100007,Lithonia Lighting Quantum 2-Light Black LED Em...,The Quantum Adjustable 2-Light LED Black Emerg...,Lithonia Lighting


In [99]:
product_all.brand = product_all.brand.apply(lambda s:s.replace('None','Unbranded'))

# merge attributes

In [45]:
df_arr = pd.read_csv('attributes.csv')
df_arr.tail(1)

,product_uid,name,value
2044802,224428,Tools Product Type,Power Tool


In [46]:
df_arr.value = df_arr.value.apply(lambda s:str(s).replace(',',' '))

In [47]:
pid = df_arr.product_uid.tolist()
values = df_arr.value.tolist()
puid = df_arr.product_uid.unique()

In [51]:
puid.size

86264

In [50]:
content = []
for i in puid:
    value = df_arr[df_arr.product_uid==i].value.tolist()
    content+=[' '.join(value)]
len(content)

86264

In [70]:
f = open('atrr_merge.csv','w')
for i in range(len(content)):
    text =str(puid[i])[:-2]+','+ content[i]+'\n'
    #print text
    #break
    f.write(text)
f.close()

# merge atrr to products

In [111]:
df_atr = pd.read_csv('atrr_merge.csv',names=['prod_id','attribute'])
df_atr.head(1)

,prod_id,attribute
0,100001,Versatile connector for various 90° connection...


In [112]:
df_atr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86263 entries, 0 to 86262
Data columns (total 2 columns):
prod_id      86263 non-null int64
attribute    86263 non-null object
dtypes: int64(1), object(1)
memory usage: 2.0+ MB


In [113]:
product_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 124428 entries, 0 to 124427
Data columns (total 4 columns):
prod_id                124428 non-null int64
product_title          124428 non-null object
product_description    124428 non-null object
brand                  124428 non-null object
dtypes: int64(1), object(3)
memory usage: 4.7+ MB


In [115]:
product_all = product_all.merge(df_atr,on='prod_id',how='left')

# pick search term

In [119]:
search = ['id','product_uid','search_term_new','relevance']
search_all = df_all[search]
search_all.shape

(240760, 4)

# output

In [116]:
product_all.to_csv('product.csv',index=False)

In [120]:
search_all.to_csv('search.csv',index=False)